In [1]:
!nvidia-smi

Sun Nov 28 10:46:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.96       Driver Version: 471.96       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:07:00.0  On |                  N/A |
|  0%   48C    P3    40W / 220W |   1614MiB /  8192MiB |     27%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# 计算设备
import torch 
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1') # 3：如果有多个GPU，可以选择

(device(type='cpu'),
 <torch.cuda.device at 0x1e8d6d6b6a0>)

In [3]:
torch.cuda.device_count()

1

In [4]:
"""
    这两个函数允许我们在请求的GPU不存在的情况下运行代码
"""
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有，则返回[cpu(),]"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())
    ]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
# 查询张量所在的设备
x = torch.tensor([1, 2, 3]) # 默认在CPU上   
x.device

device(type='cpu')

In [17]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [18]:
Y = torch.ones(2, 3, device=try_gpu(1))
Y


tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [19]:
Z = X.cuda(0) # 在同一个GPU上才能相加
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [20]:
X + Z

tensor([[2., 2., 2.],
        [2., 2., 2.]], device='cuda:0')

In [21]:
Y + Z # 从CPU到GPU的数据迁移很慢，所以干脆报错

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [23]:
Z.cuda(0) is Z # 如果已经在这块GPU上，.cuda不会拷贝新的对象

True

In [24]:
"""
    神经网络与GPU
"""
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu()) # 挪到GPU上

net(X)

tensor([[0.6453],
        [0.6453]], device='cuda:0', grad_fn=<AddmmBackward>)

In [25]:
net[0].weight.data.device

device(type='cuda', index=0)